<a href="https://colab.research.google.com/github/aleenakjames/German-Translation-using-LLMs/blob/main/German_translation_in_wikipedia_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing all the required libraries**


In [ ]:
!pip install langchain_community nltk==3.5 sacrebleu sacremoses

# **Libraries and Imports**
* WebBaseLoader from langchain_community.document_loaders is used for fetching and processing web documents. It is particularly used for retrieveing the wikipedia article being searched for.
* RecursiveCharacterTextSplitter splits long texts into smaller, manageable chunks based on character count, ensuring the chunks fit within token limits for processing by language models.
* BLEU score,Translation Edit Rate (TER), and ChrF score are the metrices for the evaluation of the translations.

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, MarianMTModel, MarianTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_metric
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time


# **Loading data from wikipedia**
The WebBaseLoader is initialized with the URL of the Wikipedia page for Koalas in German ("https://de.wikipedia.org/wiki/koala"). This sets up the loader to fetch and process the content from this URL.

In [ ]:
#load the required url
loader = WebBaseLoader("https://de.wikipedia.org/wiki/Koala")
#load the data
data = loader.load()

# **Data Preprocessing**
Unwanted characters such as newlines, pipes, slashes, parentheses, square brackets, and backslashes are removed from each chunk.The beginning and end of each chunk are trimmed to remove extraneous content. This preprocessing step ensures that the text chunks are cleaned and ready for further processing or analysis.

**Splitting large texts**

split_documents() method of the text_splitter instance to split the loaded data into smaller chunks. The data variable, which contains the text loaded from the web page, is divided into chunks based on the specified chunk_size (Each chunk will have a maximum of 2000 characters) and chunk_overlap (There will be an overlap of 200 characters between consecutive chunks. This ensures that important information at the boundaries of chunks is not lost.) The resulting chunks are stored in the docs variable.

In [ ]:
#split the data into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100, )
docs= text_splitter.split_documents(data)

In [ ]:
# Create a translation table that maps each character to None
remove_chars = "|\\/()[]"
translation_table = str.maketrans('', '', remove_chars)

In [ ]:
#remove unwanted characters in each chunk of data
for i in range(0, len(docs)):
  if docs[i] is not None:
    docs[i] = str(docs[i]).replace("\\n", "").translate(translation_table)
    docs[i] = docs[i][22:-104]

# **Inference using different models**
* **mBART:** The model can translate directly between any pair of 50 languages. To translate into a target language, the target language id is forced as the first generated token. To force the target language id as the first generated token, pass the forced_bos_token_id parameter to the generate method.
* **MarianMT:** The model "Helsinki-NLP/opus-mt-de-en" is a neural machine translation model designed specifically for translating text from German (de) to English (en), and it is part of the OPUS-MT project developed by the Helsinki-NLP group.
* **Tsmall100:** It is a compact and fast massively multilingual machine translation model covering more than 10K language pairs, that achieves competitive results while being much smaller and faster.

In [ ]:
# Models and Tokenizers
models = {
    "MarianMT": ("Helsinki-NLP/opus-mt-de-en", MarianMTModel, MarianTokenizer),
    "mBART": ("facebook/mbart-large-50-many-to-many-mmt", MBartForConditionalGeneration, MBart50TokenizerFast),
    "Tsmall100": ("alirezamsh/small100", AutoModelForSeq2SeqLM, AutoTokenizer)
}

In [ ]:
df = pd.DataFrame({})
inferenceTime = {}

In [ ]:
for model_name, (model_checkpoint, model_class, tokenizer_class) in models.items():
    # Load model and tokenizer
    print(model_name)
    model = model_class.from_pretrained(model_checkpoint)
    tokenizer = tokenizer_class.from_pretrained(model_checkpoint)
    translated = []
    #translate each chunk to english
    start_time = time.time()
    for i in range(0, len(docs)):
        if model_name == "mBART":
            tokenizer.src_lang = "de_DE"
            encoded_de = tokenizer(docs[i], return_tensors="pt")
            generated_tokens = model.generate(
                **encoded_de,
                forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
            )
            translated.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))
        elif model_name == "Tsmall100":
            encoded_de = tokenizer(docs[i], return_tensors="pt")
            generated_tokens = model.generate(**encoded_de)
            translated.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))
        else:
            encoded_de = tokenizer(docs[i], return_tensors="pt", max_length=512, truncation=True)
            generated_tokens = model.generate(**encoded_de)
            translated.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))
    translated_text = [element[0] for element in translated]
    end_time = time.time()
    inference_time = end_time - start_time
    inferenceTime[model_name] = inference_time
    df[model_name] = translated_text
    df.to_csv("translation.csv", index=False)

# **Calculating Scores**
**BLEU:** The score evaluates the quality of machine-generated translations by comparing them to reference translations. Weights define the importance of different n-grams in the BLEU score calculation.

**TER:** The Translation Edit Rate score measures the number of edits required to change a machine-translated output into one of the reference translations, with a lower score indicating a higher quality translation.

**ChrF:** The Character F-score metric is another evaluation metric for machine translation quality, which computes precision and recall over character n-grams, not word n-grams.

In [ ]:
bleu_metric = load_metric('sacrebleu', trust_remote_code=True)
ter_metric = load_metric('ter', trust_remote_code=True)
chrf_metric = load_metric('chrf', trust_remote_code=True)

In [ ]:
# Functions to evaluate translations
def evaluate_bleu(predictions, references):
    return bleu_metric.compute(predictions=predictions, references=[[ref] for ref in references])
def evaluate_chrf(predictions, references):
    return chrf_metric.compute(predictions=[pred.split() for pred in predictions], references=[[ref.split()] for ref in references])
def evaluate_ter(predictions, references):
    return ter_metric.compute(predictions=[pred.split() for pred in predictions], references=[[ref.split()] for ref in references])

In [ ]:
results = {}
keys = list(models.keys())

# Calculate all pairwise comparisons
for i in range(len(keys)):
    for j in range(len(keys)):
        if i != j:
            bleu = evaluate_bleu(df[keys[i]], df[keys[j]])
            ter = evaluate_ter(df[keys[i]], df[keys[j]])
            chrf = evaluate_chrf(df[keys[i]], df[keys[j]])
            results[(keys[i], keys[j])] = {
                "BLEU": bleu,
                "TER": ter,
                "ChrF": chrf
            }

In [ ]:
for model, time_s in inferenceTime.items():
        print(f"Inferece time of {model}: {time_s}")
for (model1, model2), result in results.items():
    print(f"Model: {model1} to {model2}")
    for metric, score in result.items():
        print(f"  {metric}: {score}")

# **Visualising the results**

In [ ]:
# Initialize dictionaries to hold DataFrames for each metric
metrics = ['BLEU', 'TER', 'ChrF']
metric_dfs = {metric: pd.DataFrame(index=keys, columns=keys) for metric in metrics}

# Populate the DataFrames with the 'score' from each metric's dictionary
for (model1, model2), scores in results.items():
    for metric in metrics:
        metric_dfs[metric].at[model1, model2] = scores[metric]['score']

In [ ]:
def plot_heatmap(df, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(df.astype(float), annot=True, cmap='coolwarm', fmt='.2f')
    plt.title(title)
    plt.show()

In [ ]:
# Plot the results for each metric
for metric in metrics:
    plot_heatmap(metric_dfs[metric], f'{metric} Scores')